# 00 - Environment Setup

Se presenta a continuación el "notebook" para desarrollar el proyecto de Grupo 3 de la materia Sistemas en la Nube PAO 1 2024 ESPOL. 

"Sistema de aprendizaje automático para la detección temprana y precisa de enfermedades fúngicas en cultivos de papa, fresa y cereza en la sierra de Ecuador". 

Integrantes:
- Jara Yupa Ana Belén
- Rengifo Barciona Pedro Francisco
- Quizhpi Torres Beatriz Aurora



---
## Setup

En esta sección se realizan pre-requisitos para desarrollar el proyecto, tales como variables globales, importar librerias y activar los clientes.

Obtener Nombre Mi Proyecto

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'grupo03-ml00'

In [ ]:
Establecer la región de trabajo

In [2]:
REGION = 'us-central1'

packages:

In [3]:
from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

clients:

In [4]:
gcs = storage.Client(project = PROJECT_ID)

parameters:

In [5]:
BUCKET = PROJECT_ID

---
## 1. Crear Bucket de Almacenamiento
Inicialmente revisa si está creado, y si no lo crea:
- [GCS Python Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [6]:
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

Created Bucket: grupo03-ml00


In [8]:
print(f'Revisar el bucket creado aqui:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}')

Revisar el bucket creado aqui:
https://console.cloud.google.com/storage/browser/grupo03-ml00;tab=objects&project=grupo03-ml00


---
<a id = 'permissions'></a>
## 2. Service Account & Permissions

Esta instancia de notebook se ejecuta como una cuenta de servicio en GCP. Esta cuenta de servicio también se utilizará para ejecutar otros servicios en Vertex AI, como training jobs y pipelines. La cuenta de servicio necesitará permiso para interactuar con el objeto en Cloud Storage que requiere la función.([roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles)).  

Get the current service account:

In [9]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'646324389573-compute@developer.gserviceaccount.com'

Enable the Cloud Resource Manager API:

In [10]:
!gcloud services enable cloudresourcemanager.googleapis.com

List the service accounts current roles:

In [14]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/editor
roles/owner
roles/run.admin
roles/storage.objectAdmin


Si en la lista resultante falta `roles/storage.objectAdmin` u otra función que contenga este permiso, como la función básica `roles/owner`, entonces será necesario agregarlo para la cuenta de servicio. Utilice estas instrucciones para completar los roles:

In [18]:
print(f'Go To IAM in the Google Cloud Console:\nhttps://console.cloud.google.com/iam-admin/iam?orgonly=true&project={PROJECT_ID}&supportedpurview=organizationId')

Go To IAM in the Google Cloud Console:
https://console.cloud.google.com/iam-admin/iam?orgonly=true&project=statmike-mlops-349915&supportedpurview=organizationId


Desde el enlace de la consola anterior, o accediendo a https:/console.cloud.google.com y navegando a "IAM y administrador > IAM":
- Localice la fila de la cuenta de servicio que aparece arriba: `<número de proyecto>-compute@developer.gserviceaccount.com`
- Debajo de la columna "herencia", haga clic en el ícono de lápiz para editar roles
- En el menú desplegable, en "Asignar roles", seleccione "Agregar otro rol".
- Haga clic en el cuadro "Seleccionar una función" y escriba "Administrador de objetos de almacenamiento", luego seleccione "Administrador de objetos de almacenamiento".
- Haga clic en Guardar
- Vuelva a ejecutar la lista de servicios a continuación y verifique que se haya agregado la función:

In [16]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/storage.objectAdmin


---
## OPCIONAL Cargar data al bucket desde Github (code)
Tambien existe la opción de insertar directamente al bucket, la solución a continuacion se plantea mediante un repositorio en github.
Creará la carpeta y copiara los archivos desde el repositorio, este proceso requerira tiempo ya que el data consiste en 3 plantas: "potato", "strawberry" y "cherry" con un total de 5623 imagenes. La data se obtuvo de kaggle y de ahi se selecciono 3 plantas para minimizar el procesamiento 
Fuente Kaggle PlantVillageDiseased https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset


In [46]:
github_repo = 'pfrengifob/SN_2024PAO1'
github_path = 'data'
gcs_data_folder = 'data'
csv_file_path = 'vegeCsv.csv'  # Ruta en el entorno de Jupyter

In [18]:
import requests
import os
import csv


In [35]:
# Funcion Listar archivos de github recursivamente
def list_github_files(repo, path):
    url = f'https://api.github.com/repos/{repo}/contents/{path}'
    response = requests.get(url)
    response.raise_for_status()
    contents = response.json()
    
    files = []
    for content in contents:
        if content['type'] == 'file' and content['name'].lower().endswith(('jpg', 'jpeg', 'png')):
            files.append(content)
        elif content['type'] == 'dir':
            files.extend(list_github_files(repo, content['path']))
    
    return files

In [36]:
# Funcion para descargar los archivos y subirlos al bucket de GCS (GoogleCloudStorage)
def download_and_upload_file(bucket_name, github_file, gcs_folder):
    file_url = github_file['download_url']
    file_path = github_file['path']
    local_path = os.path.join('/tmp', os.path.basename(file_path))
    
    # Descargar Archivo
    response = requests.get(file_url)
    response.raise_for_status()
    with open(local_path, 'wb') as f:
        f.write(response.content)
    
    # Subir Archivo a GCS
    gcs_path = file_path.replace("\\", "/")  # Emplea Misma Ruta que Github
    bucket = gcs.bucket(bucket_name)
    blob = bucket.blob(gcs_path)
    blob.upload_from_filename(local_path)
    print(f'Uploaded {local_path} to gs://{bucket_name}/{gcs_path}')

In [37]:
# Llamar a listar archivos de Github
github_files = list_github_files(github_repo, github_path)

In [ ]:
# Descargar y cargar cada archivo siguiendo la misma estructura
for github_file in github_files:
    download_and_upload_file(BUCKET, github_file, gcs_data_folder)

---
## 3. Preparar Data
Se requiere preparar datos de entrenamiento de imágenes para usarlos en un conjunto de datos de Vertex AI a fin de entrenar un modelo de clasificación de imágenes. Para ello hay que seguir la estructura que indican en la documentación: https://cloud.google.com/vertex-ai/docs/image-data/classification/prepare-data?hl=es-419#csv

Se empleará el formata csv con las columnas [ML_USE],GCS_FILE_PATH,[LABEL]

Lista de columnas

ML_USE (opcional): Para fines de división de datos durante el entrenamiento de un modelo. Usa TRAINING, TEST o VALIDATION. En este proyecto se considera el 80% para Training, 10% Test y 10% Validation.

GCS_FILE_PATH: este campo contiene el URI de Cloud Storage para la imagen. Los URI de Cloud Storage distinguen entre mayúsculas y minúsculas.

LABEL (opcional): las etiquetas deben comenzar con una letra y solo deben contener letras, números y guiones bajos.
Ejemplo de CSV: image_classification_single_label.csv


In [53]:
import random

In [42]:
# Funcion Listar imagenes de GoogleCloudStorage y obetener URIs
def list_images_in_gcs(bucket_name, gcs_folder):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=gcs_folder)
    
    gcs_uris = []
    for blob in blobs:
        if blob.name.lower().endswith(('jpg', 'jpeg', 'png')):
            label = blob.name.split('/')[-2]  # Assuming the label is the parent folder name
            gcs_uri = f'gs://{bucket_name}/{blob.name}'
            gcs_uris.append((gcs_uri, label))
            print(f'Found {gcs_uri} with label {label}')
    
    return gcs_uris

In [51]:
# Funcion Generar csv a partir de URIs
def generate_csv_from_gcs_uris(gcs_uris, csv_file_path):
    # Mezclar la lista
    random.shuffle(gcs_uris)
    
    # Split 
    train_split = int(0.8 * len(gcs_uris))
    val_split = int(0.9 * len(gcs_uris))
    
    train_uris = gcs_uris[:train_split]
    val_uris = gcs_uris[train_split:val_split]
    test_uris = gcs_uris[val_split:]
    
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ML_USE', 'GCS_FILE_PATH', 'LABEL'])
        
        for gcs_uri, label in train_uris:
            writer.writerow(['training', gcs_uri, label])
        
        for gcs_uri, label in val_uris:
            writer.writerow(['validation', gcs_uri, label])
        
        for gcs_uri, label in test_uris:
            writer.writerow(['test', gcs_uri, label])
    
    print(f'CSV file created at {csv_file_path} with {len(gcs_uris)} images.')

In [ ]:
# Llamar a la funcion listar imagenes
gcs_uris = list_images_in_gcs(BUCKET, gcs_data_folder)

In [54]:
# Generar csv
generate_csv_from_gcs_uris(gcs_uris, csv_file_path)

CSV file created at vegeCsv.csv with 5462 images.


---
## 4. Cargar Dataset (Consola)
A continuación se aprovecha la interfaz de GCP y se carga el respectivo .CSV en la mismo ubicacion donde se encuentre nuestro bucket (Desplegamos Menú Navegación de Apis, buscamos Cloud Storage y seleccionamos Buckets). 
De esta manera no habrá conflicto para realizar lectura de datos.



![CSV Bucket](./imagenes/csvBucket.png)

Posteriormente se crea el conjunto de datos en Vertex AI (Desplagamos Menú Izquierda - Vertex AI Api - Data - Conjunto de Datos). Se selecciona Clasificación con varias etiquetas y la misma región que estamos desarrollando. 



![CSV Bucket](./imagenes/plantDataset.png)

Se procede a dar click en el dataset creado e importamos el csv indicando la ruta.



![CSV Bucket](./imagenes/plantDataset2.png)

Este proceso requiere unos minutos y una vez culminado tendremos un breve resumen de nuestros datos etiquetados listos para entrenar, además de la cantidad para entrenamiento, validación y test, tal como se muestra a continuación:



<div style="display: flex; justify-content: center;">
    <img src="./imagenes/plantDataset3.png" alt="CSV Bucket 1" style="margin-right: 10px; width: 400px;"/>
    <img src="./imagenes/plantDataset4.png" alt="width: 300px;"/>
</div>

---
## 5. Entrenar Modelo e Implementar en el Extremo (Consola)
Con nuestro dataset listo, se emplea AutoML como modelo de gran calidad y minimo esfuerzo de machine learning, además se especifican la cantidad de nodos (9 por hora) para el entrenamiento. Para ello en la misma sección del paso previo, se selecciona "Entrenar Nuevo Modelo". 



![CSV Bucket](./imagenes/training.png)

El entrenamiento dependerá de la cantidad de nodos asignados y una vez culminado notificará por correo, para este proyecto demoró 1 hora y 48 minutos con 9 nodos. Adicionalmente se puede apreciar detalles del modelo y estadísticas como la evaluación realizada, la precisión y el limite de confianza.



![CSV Bucket](./imagenes/training2.png)



![CSV Bucket](./imagenes/training3.png)

Finalmente, en la seccion "Deploy And Use" de Vertex AI, seleccionamos Prediccion en linea y creamos un "Extremo" con un nodo de procesamiento. El Endpoint nos permitira realizar peticiones ya sea en la plataforma, mediante Rest o Python, de esa manera se logrará predecir el estado del vegetal.



![CSV Bucket](./imagenes/endpoint.png)

---
## 6. Predicciones
Una vez implementado con exito el modelo de deteccion de estado de fresas, cerezas y papas, a partir de peticiones en Rest o Python se pueden realizar distintos ensayos para simular en un entorno real.  

inputs

In [71]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
import base64

In [69]:
#Funcion predecir imagenes 
def predict_image_classification_sample(
    project: str,
    endpoint_id: str,
    filename: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    
    # Read the image file
    with open(filename, "rb") as f:
        file_content = f.read()

    # Encode the image content
    encoded_content = base64.b64encode(file_content).decode("utf-8")
    instance = predict.instance.ImageClassificationPredictionInstance(
        content=encoded_content,
    ).to_value()
    instances = [instance]
    
    # Set the parameters for the prediction
    parameters = predict.params.ImageClassificationPredictionParams(
        confidence_threshold=0.5,
        max_predictions=5,
    ).to_value()
    
    # Get the endpoint path
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    
    # Make the prediction request
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    
    # Print the response
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    
    # Print the predictions
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

In [92]:
#Llamar a predecir cherry
predict_image_classification_sample(
    project="646324389573",
    endpoint_id="5497207394670739456",
    filename="./imageTest/cherry3.JPG"
)

response
 deployed_model_id: 8142166178169618432
 prediction: {'displayNames': ['Cherry_(including_sour)___Powdery_mildew'], 'confidences': [0.934258], 'ids': ['8748327213440434176']}



![CSV Bucket](./imageTest/cherry3.JPG)

In [91]:
#Llamar a predecir potato
predict_image_classification_sample(
    project="646324389573",
    endpoint_id="5497207394670739456",
    filename="./imageTest/potato4.JPG"
)

response
 deployed_model_id: 8142166178169618432
 prediction: {'displayNames': ['Potato___healthy'], 'confidences': [0.985289335], 'ids': ['1830798185799352320']}


![CSV Bucket](./imageTest/potato4.JPG)

In [88]:
#Llamar a predecir strawberry
predict_image_classification_sample(
    project="646324389573",
    endpoint_id="5497207394670739456",
    filename="./imageTest/strawberry2.jpg"
)

response
 deployed_model_id: 8142166178169618432
 prediction: {'displayNames': ['Strawberry___Leaf_scorch'], 'ids': ['389646305040793600'], 'confidences': [0.908504546]}


![CSV Bucket](./imageTest/strawberry2.jpg)


---
## 7. Resultados
Se logró predecir con gran porcentaje de aceptación en los 3 distintos ensayos con más de 90% de certeza el estado de la planta, además de identificar su tipo (frambuesas, papas y cerezas). 
Además, se logró usar predicciones en línea por medio de Python y Jupyter incorporados como clientes de Google Cloud Platform.